In [ ]:
from google.colab import files
import pandas as pd
import torch
from torch.optim import AdamW
from transformers import BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from tqdm import tqdm



In [ ]:


file_path= r'/content/cleaned_data.csv'

# Read the file into a DataFrame
df = pd.read_csv(file_path)

# Now you can work with the DataFrame 'df'
# For example, display the first few rows
df.head()

,Insult,Comment,cleaned_comment,tokenized-comment
0,1,"""You fuck your dad.""",fuck dad,"['fuck', 'dad']"
1,0,"""i really don't understand your point.\xa0 It ...",really dont understand pointxa seems mixing ap...,"['really', 'dont', 'understand', 'pointxa', 's..."
2,0,"""A\\xc2\\xa0majority of Canadians can and has ...",Axcxamajority Canadians wrong againnnUnless yo...,"['Axcxamajority', 'Canadians', 'wrong', 'again..."
3,0,"""listen if you dont wanna get married to a man...",listen dont wan na get married man women DONT ...,"['listen', 'dont', 'wan', 'na', 'get', 'marrie..."
4,0,"""@SDL OK, but I would hope they'd sign him to ...",SDL OK would hope theyd sign oneyear contract ...,"['SDL', 'OK', 'would', 'hope', 'theyd', 'sign'..."


In [ ]:
texts = df['cleaned_comment'].values
labels = df['Insult'].values

# Ensure that each item in 'texts' is a string
texts = [str(text) for text in df['cleaned_comment'].values]



In [ ]:
# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_texts = [tokenizer(text, padding='max_length', max_length=256, truncation=True, return_tensors="pt") for text in texts]

In [ ]:
# Example text
sample_text = texts[0]

# Tokenize the sample text
tokenized_example = tokenizer(sample_text, padding='max_length', max_length=256, truncation=True, return_tensors="pt")

# Print the tokenized example
print("Tokenized Example:", tokenized_example)




Tokenized Example: {'input_ids': tensor([[ 101, 6616, 3611,  102,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  

In [ ]:
# Check if 'labels' is already a tensor
if isinstance(labels, torch.Tensor):
    # Clone and detach if it's already a tensor
    labels_tensor = labels.clone().detach()
else:
    # Otherwise, convert it to a tensor
    labels_tensor = torch.tensor(labels, dtype=torch.long)

# Creating tensors for texts
input_ids = torch.cat([x['input_ids'] for x in tokenized_texts], dim=0)
attention_masks = torch.cat([x['attention_mask'] for x in tokenized_texts], dim=0)

# Create TensorDataset using the cloned/detached labels tensor
dataset = TensorDataset(input_ids, attention_masks, labels_tensor)


In [ ]:
import pandas as pd

# Extracting tensors from the dataset
input_ids, attention_masks, labels = zip(*[dataset[i] for i in range(len(dataset))])

# Converting tensors to lists
# Here, we'll convert only a part of 'input_ids' and 'attention_masks' for demonstration
input_ids_list = [ids[0:10].tolist() for ids in input_ids]  # First 10 tokens
attention_masks_list = [masks[0:10].tolist() for masks in attention_masks]  # First 10 tokens
labels_list = [label.item() for label in labels]

# Create a DataFrame
df = pd.DataFrame({
    'Input IDs': input_ids_list,
    'Attention Masks': attention_masks_list,
    'Labels': labels_list
})
df.head()


,Input IDs,Attention Masks,Labels
0,"[101, 6616, 3611, 102, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0]",1
1,"[101, 2428, 2123, 2102, 3305, 2391, 18684, 384...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0
2,"[101, 22260, 2278, 18684, 2863, 5558, 15780, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0
3,"[101, 4952, 2123, 2102, 14071, 6583, 2131, 249...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0
4,"[101, 17371, 2140, 7929, 2052, 3246, 2027, 209...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0


In [ ]:
# Splitting dataset into training and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Initializing the BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.cuda()  # Move model to GPU if available

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
epochs = 6
# Setting up optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [ ]:

# Directly assign the model to GPU
device = torch.device("cuda")
model.to(device)

# Training loop
epochs = 4
best_accuracy = 0.0
best_epoch = 0
for epoch in range(epochs):
    model.train()
    total_loss = 0
    predictions, true_labels = [], []
    for step, batch in enumerate(train_dataloader):
        b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)
        model.zero_grad()
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        logits = outputs.logits
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend(np.argmax(logits, axis=1).flatten())
        true_labels.extend(label_ids.flatten())

    avg_train_loss = total_loss / len(train_dataloader)
    train_accuracy = accuracy_score(true_labels, predictions)
    print(f'Epoch {epoch + 1}, Average Training Loss: {avg_train_loss}, Training Accuracy: {train_accuracy}')

    # Check if this is the best accuracy so far
    if train_accuracy > best_accuracy:
        best_accuracy = train_accuracy
        best_epoch = epoch + 1
    else:
        # Stop training if the model is not improving
        print(f'No improvement in accuracy after epoch {epoch + 1}. Stopping training.')
        break

print(f'Best Training Accuracy: {best_accuracy} at Epoch {best_epoch}')


Epoch 1, Average Training Loss: 0.4449277873517889, Training Accuracy: 0.7991128010139417
Epoch 2, Average Training Loss: 0.28556786270605194, Training Accuracy: 0.8868821292775665
Epoch 3, Average Training Loss: 0.18615796281532807, Training Accuracy: 0.9347275031685678
Epoch 4, Average Training Loss: 0.1002330687773094, Training Accuracy: 0.9702154626108999
Best Training Accuracy: 0.9702154626108999 at Epoch 4


In [ ]:
def classify_insult_bert(sentence):
    # Tokenize the input sentence
    inputs = tokenizer.encode_plus(
        sentence,
        None,
        add_special_tokens=True,
        max_length=256,
        padding='max_length',
        return_token_type_ids=False,
        return_attention_mask=True,
        truncation=True,
        return_tensors='pt'
    )

    # Move tensors to the GPU
    input_ids = inputs['input_ids'].to('cuda')
    attention_mask = inputs['attention_mask'].to('cuda')

    # Get prediction
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        prediction = torch.argmax(outputs.logits, dim=1)

    if prediction.item() == 1:
        print("1")  # Insult
    else:
        print("0")  # Not an insult

# Test the function with different sentences
classify_insult_bert("fuck you both")
classify_insult_bert("go kill yourself")
classify_insult_bert("I love this!!!.")
classify_insult_bert("go fuck your dad")
classify_insult_bert("this is a good day.")
classify_insult_bert("get cancer.")

1
1
0
1
0
1
